In [93]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import openml
import seaborn as sn

In [129]:
def get_dataset(id):

    suite = openml.study.get_suite(99)
    
    task = openml.tasks.get_task(id)
    dataset = openml.datasets.get_dataset(dataset_id,)
    
    X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="array", target=dataset.default_target_attribute
    )
    
    df = pd.DataFrame(X, columns=attribute_names)
    # converter:
    #   0 -> -1
    #   1 -> 1
    df['target'] = 2*y-1
    # erase rows with NaN values
    df = df.dropna(how='any', axis=0)
    
    
    '''plt.matshow(df.corr())
    plt.show()
    corr = df.corr()
    corr.style.background_gradient(cmap='Purples')
    # 'RdBu_r', 'BrBG_r', & PuOr_r are other good diverging colormaps
    '''
    heatMap = df.corr()
    sn.heatmap(heatMap, cmap=sn.cubehelix_palette(rot=.2))
    
    return df

In [132]:
list = [37, 1461, 61, 13, 31, 1464, 4534, 4134, 885, 40994]
list_num_dataset_id = [1464, 1067, 1467, 1504,1494, 1068]
dataframes = {}
for dataset_id in list_num_dataset_id:
    df = get_dataset(dataset_id)
    dataframes[dataset_id] = df

C:\Users\ricar\AppData\Local\Temp\ipykernel_24424\3528415745.py:5: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  df = get_dataset(dataset_id)


OpenMLServerException: https://www.openml.org/api/v1/xml/task/1464 returned code 151: Unknown task - None

In [133]:
dataframes.keys()

dict_keys([])

In [114]:
import openml
import pandas as pd

# Definindo os IDs dos conjuntos de dados
dataset_ids = [13, 31, 1461]

# Criando um dicionário para armazenar os dataframes
dataframes = {}

# Loop para baixar os datasets e converter em dataframes
for dataset_id in dataset_ids:
    dataset = openml.datasets.get_dataset(dataset_id, download_data=True, download_qualities=True, download_features_meta_data=True)
    df, *_ = dataset.get_data(target=dataset.default_target_attribute)
    dataframes[dataset_id] = df

le = LabelEncoder()
for id in dataset_ids:
    dataframes[id].rename(columns={dataframes[id].columns[-1]: 'target'}, inplace=True)

    dataframes[id]['target'] = le.fit_transform(dataframes[id]['target'])
    dataframes[id]['target'] = dataframes[id]['target'].apply(lambda x: 1 if x == 1 else -1)

    dataframes[id]  = dataframes[id].dropna()
# Salvando os dataframes como arquivos CSV
for dataset_id, df in dataframes.items():
    df.to_csv(f'dataset_{dataset_id}.csv', index=False)


In [113]:
dataframes[31]

dict_keys([13, 31])

In [84]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold



def run_cv(X,y,algs,nfolds=10, means_only=False):
    results = {}
    kf = KFold(n_splits=nfolds, shuffle=True, random_state=1111)
    for algo_name, algo in algs:
        results[algo_name] = []
        for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            algo.fit(X_train, y_train, M=40)
            y_pred = algo.predict(X_test)
            results[algo_name].append(accuracy_score(y_test, y_pred))
    results_df = pd.DataFrame.from_dict(results)
    if not means_only:
        return results_df
    else:
        results_means = {}
        for algo_name, algo in algs:
            results_means[algo_name] = [np.mean(results[algo_name])]
        return pd.DataFrame.from_dict(results_means)

In [85]:
import matplotlib.pyplot as plt

def plot_cv(results_cv,metric='Accuracy', title="Cross-validation results for multiple algorithms in a single task"):
    fig, ax = plt.subplots()
    ax.boxplot(results_cv)
    ax.set_xticklabels(results_cv.columns)
    ax.set_ylabel(metric)
    ax.set_title(title)
    plt.show()

In [89]:
from algorithms.adaboost import AdaBoost
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
models = [
    ("AdaBoost Default", AdaBoost()),
    #("AdaBoost KNN", AdaBoost(knn=True))
]

for dataset_id in dataset_ids:
    X = dataframes[dataset_id].drop(columns=['target'], axis=1)
    y = dataframes[dataset_id]['target']
    # Dividindo os dados em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Criando e treinando o modelo AdaBoost
    model = AdaBoost()
    model.fit(X_train, y_train)

    # Testando o modelo
    y_pred = model.predict(X_test)

    # Calculando a acurácia
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Acurácia do modelo para dataset {dataset_id}: {accuracy}")

ValueError: could not convert string to float: '60-69'

In [91]:
dataset[0]

TypeError: 'OpenMLDataset' object is not subscriptable